<a href="https://colab.research.google.com/github/imabari/covid19-data/blob/master/vaccine/%E5%89%AF%E5%8F%8D%E5%BF%9C%E7%96%91%E3%81%84%E5%A0%B1%E5%91%8A%E9%9B%86%E8%A8%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://www.mhlw.go.jp/content/10906000/000784435.pdf -O data.pdf

--2021-05-27 08:11:24--  https://www.mhlw.go.jp/content/10906000/000784435.pdf
Resolving www.mhlw.go.jp (www.mhlw.go.jp)... 23.77.29.154, 2600:1417:8000:c93::3194, 2600:1417:8000:caa::3194
Connecting to www.mhlw.go.jp (www.mhlw.go.jp)|23.77.29.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4932198 (4.7M) [application/pdf]
Saving to: ‘data.pdf’

data.pdf            100%[===================>]   4.70M  22.4MB/s    in 0.2s    

2021-05-27 08:11:25 (22.4 MB/s) - ‘data.pdf’ saved [4932198/4932198]



In [2]:
!apt install python3-tk ghostscript
!pip install camelot-py[cv]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-tk is already the newest version (3.6.9-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono gsfonts libcupsfilters1 libcupsimage2
  libgs9 libgs9-common libijs-0.35 libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto ghostscript-x poppler-utils fonts-japanese-mincho
  | fonts-ipafont-mincho fonts-japanese-gothic | fonts-ipafont-gothic
  fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gsfonts libcupsfilters1
  libcupsimage2 libgs9 libgs9-common libijs-0.35 libjbig2dec0 poppler-data
0 upgraded, 11 newly installed, 0 to remove and 34 not upgraded.
Need to get 14.1 MB of archives.
After this operation, 

In [3]:
import camelot
import pandas as pd

In [5]:
from google.colab import files

# PDF変換

In [4]:
tables = camelot.read_pdf("data.pdf", pages="2-end", split_text=True)

df = pd.concat([table.df for table in tables])

df.to_csv("data.csv", encoding="utf_8_sig", index=False, header=False)

# ダウンロード

In [6]:
files.download("data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 前処理

In [15]:
import re
import pandas as pd

In [7]:
df0 = pd.read_csv("data.csv", dtype="object").fillna("")

In [8]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7298 entries, 0 to 7297
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   No             7298 non-null   object
 1   年齢             7298 non-null   object
 2   性別             7298 non-null   object
 3   接種日            7298 non-null   object
 4   発生日            7298 non-null   object
 5   接種から
発生までの
日数  7298 non-null   object
 6   ワクチン名          7298 non-null   object
 7   製造販売業者         7298 non-null   object
 8   ロット番
号         7298 non-null   object
 9   症状名（PT名）       7298 non-null   object
 10  因果関係
（報告医評価）   7298 non-null   object
 11  重篤度
（報告医評価）    7298 non-null   object
 12  転帰日            7298 non-null   object
 13  転帰内容           7298 non-null   object
dtypes: object(14)
memory usage: 798.3+ KB


In [9]:
df0.columns = df0.columns.str.replace("\n", "")

In [10]:
df0["No"] = df0["No"].str.replace("※[１-９]", "").str.strip().astype(int)

In [11]:
df0["年齢"] = df0["年齢"].str.rstrip("歳")
df0["年齢"] = df0["年齢"].mask(df0["年齢"] == "").astype(float).astype("Int64")

In [12]:
df0["性別"] = df0["性別"].replace({"": "不明"})

# 整然データ

かのえ／Pattie／柿人 さんのコードを参考にさせていただきました

https://twitter.com/kanoekakihito/status/1397222333505486849

https://github.com/kanoekakihito/tidy_mhlw_data/blob/main/20210523_%E6%96%B0%E5%9E%8B%E3%82%B3%E3%83%AD%E3%83%8A%E3%83%AF%E3%82%AF%E3%83%81%E3%83%B3%E5%89%AF%E5%8F%8D%E5%BF%9CCSV%E6%95%B4%E5%BD%A2.py

In [13]:
df1 = df0.copy()

In [16]:
df1["症状名（PT名）"] = df1["症状名（PT名）"].apply(lambda s: [i.replace("\n", "").strip() for i in re.split("(?<=[）)]\n)", s)])
df1["発生日"] = df1["発生日"].str.split()
df1["転帰日"] = df1["転帰日"].str.split()
df1["転帰内容"] = df1["転帰内容"].str.split()

In [17]:
dfs = []

for i, row in df1.iterrows():
    dfs.append(row.apply(pd.Series).T.fillna(method="ffill"))

In [55]:
df2 = pd.concat(dfs).reset_index(drop=True)

In [56]:
df3 = df2["症状名（PT名）"].str.extract("(.+)[（()](.+?)[）)]").rename(columns={0:"症状名", 1:"PT名"})
df3["PT名"] = df3["PT名"].str.split("|")

In [57]:
df4 = df2.join(df3).explode("PT名")

In [58]:
df4["症状名"] = df4["症状名"].str.strip()
df4["PT名"] = df4["PT名"].str.strip()

In [59]:
df4["No"] = df2["No"].astype(int)
df4["年齢"] = df2["年齢"].astype("Int64")

In [60]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16742 entries, 0 to 16295
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   No           16742 non-null  int64 
 1   年齢           16740 non-null  Int64 
 2   性別           16742 non-null  object
 3   接種日          16742 non-null  object
 4   発生日          16742 non-null  object
 5   接種から発生までの日数  16742 non-null  object
 6   ワクチン名        16742 non-null  object
 7   製造販売業者       16742 non-null  object
 8   ロット番号        16742 non-null  object
 9   症状名（PT名）     16742 non-null  object
 10  因果関係（報告医評価）  16742 non-null  object
 11  重篤度（報告医評価）   16742 non-null  object
 12  転帰日          16742 non-null  object
 13  転帰内容         16742 non-null  object
 14  症状名          16742 non-null  object
 15  PT名          16742 non-null  object
dtypes: Int64(1), int64(1), object(14)
memory usage: 2.2+ MB


In [61]:
df4

,No,年齢,性別,接種日,発生日,接種から発生までの日数,ワクチン名,製造販売業者,ロット番号,症状名（PT名）,因果関係（報告医評価）,重篤度（報告医評価）,転帰日,転帰内容,症状名,PT名
0,1,55,女性,2021/03/19,2021/03/19,0,コミナティ筋注,ファイザー,EP9605,口腔咽頭不快感（口腔咽頭不快感）,,重くない,未記入,不明,口腔咽頭不快感,口腔咽頭不快感
1,1,55,女性,2021/03/19,2021/03/19,0,コミナティ筋注,ファイザー,EP9605,皮疹・発疹・紅斑（紅斑）,,重くない,未記入,軽快,皮疹・発疹・紅斑,紅斑
2,2,24,女性,2021/04/01,2021/04/01,0,コミナティ筋注,ファイザー,ER2659,悪心・嘔吐（悪心）,,重くない,未記入,軽快,悪心・嘔吐,悪心
3,2,24,女性,2021/04/01,2021/04/01,0,コミナティ筋注,ファイザー,ER2659,口腔咽頭不快感（口腔咽頭不快感）,,重くない,未記入,軽快,口腔咽頭不快感,口腔咽頭不快感
4,3,77,男性,2021/04/19,2021/04/23,4,コミナティ筋注,ファイザー,ET9096,脳出血（脳幹出血）,,重い,未記入,死亡,脳出血,脳幹出血
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16291,7297,66,女性,2021/05/15,2021/05/15,0,コミナティ筋注,ファイザー,EW4811,胸部不快感（胸部不快感）,関連あり,重くない,2021/05/15,回復,胸部不快感,胸部不快感
16292,7297,66,女性,2021/05/15,2021/05/15,0,コミナティ筋注,ファイザー,EW4811,動悸（動悸）,関連あり,重くない,2021/05/15,回復,動悸,動悸
16293,7297,66,女性,2021/05/15,2021/05/15,0,コミナティ筋注,ファイザー,EW4811,口の錯感覚（口の錯感覚）,関連あり,重くない,2021/05/15,回復,口の錯感覚,口の錯感覚
16294,7297,66,女性,2021/05/15,2021/05/15,0,コミナティ筋注,ファイザー,EW4811,皮疹・発疹・紅斑（紅斑）,関連あり,重くない,2021/05/15,回復,皮疹・発疹・紅斑,紅斑


In [50]:
df4.to_csv("output.csv", index=False, encoding="utf_8_sig")

# ダウンロード

In [ ]:
files.download("output.csv")

# 症状別報告件数

In [52]:
df4["症状"] = df4["症状名"] + "（" + df4["PT名"] + "）"

In [53]:
df4["症状"].value_counts()

発熱（発熱）                  2254
倦怠感（倦怠感）                1494
頭痛（頭痛）                  1374
アナフィラキシー（アナフィラキシー反応）     984
悪寒（悪寒）                   662
                        ... 
脊髄炎（脊髄炎）                   1
脳ヘルニア（脳ヘルニア）               1
末梢循環不全（末梢循環不全）             1
平衡障害（平衡障害）                 1
アトピー性皮膚炎（アトピー性皮膚炎）         1
Name: 症状, Length: 547, dtype: int64

# 集計確認

In [25]:
df1 = df0.copy()
df1.set_index("No", inplace=True)

## アナフィラキシー

In [26]:
df1["症状名（PT名）"].str.contains("アナフィラキシー").sum()

994

## 性別

In [27]:
# 性別 副反応疑い報告数
df1["性別"].value_counts()

女性    6048
男性    1241
不明       9
Name: 性別, dtype: int64

In [28]:
# 性別 重篤報告数
df1.loc[df1["重篤度（報告医評価）"] == "重い", "性別"].value_counts()

女性    705
男性    139
不明      2
Name: 性別, dtype: int64

In [29]:
# 性別 死亡報告数
df1.loc[df1["転帰内容"].str.contains("死亡"), "性別"].value_counts()

女性    29
男性    23
Name: 性別, dtype: int64

## 年齢別

In [30]:
labels = [
    "0～9歳",
    "10～19歳",
    "20～29歳",
    "30～39歳",
    "40～49歳",
    "50～59歳",
    "60～69歳",
    "70～79歳",
    "80歳以上",
]

In [31]:
df1["年代"] = pd.cut(
    df1["年齢"],
    [0, 10, 20, 30, 40, 50, 60, 70, 80, 120],
    include_lowest=True,
    right=False,
    labels=labels,
).astype(str)

In [32]:
df1["年代"] = df1["年代"].mask(df1["年齢"].isnull(), "不明")

In [33]:
# 年齢別副反応疑い報告数
df1["年代"].value_counts().reindex(labels + ["不明"]).fillna(0).astype(int)

0～9歳         0
10～19歳      41
20～29歳    1491
30～39歳    1724
40～49歳    2063
50～59歳    1363
60～69歳     403
70～79歳      90
80歳以上      121
不明           2
Name: 年代, dtype: int64

In [34]:
# 年齢別重篤報告数
df1.loc[df1["重篤度（報告医評価）"] == "重い", "年代"].value_counts().reindex(labels + ["不明"]).fillna(0).astype(int)

0～9歳        0
10～19歳      5
20～29歳    149
30～39歳    197
40～49歳    246
50～59歳    152
60～69歳     38
70～79歳     20
80歳以上      37
不明          2
Name: 年代, dtype: int64

In [35]:
# 年齢別死亡報告数
df1.loc[df1["転帰内容"].str.contains("死亡"), "年代"].value_counts().reindex(labels + ["不明"]).fillna(0).astype(int)

0～9歳       0
10～19歳     0
20～29歳     3
30～39歳     1
40～49歳     5
50～59歳     4
60～69歳     6
70～79歳    10
80歳以上     23
不明         0
Name: 年代, dtype: int64